#  Simulation Runner for SparkShark AMMPS Simulation - v0.0.1

## Aim  
We will test that the ammps market can produce results that are in agreement with the Lucas asset pricing model. When the market makers have market maker Lucas factor = 1 they will only consider the dividend for pricing, while if it's 0 they will only consider their inventory. moving the factor away from 1 should create deviations from the Lucas pricing model. Even when the factor is 0 we will expect some degree of alignment between the prices and the Lucas pricing model as the institutional investors are using the Lucas pricing model too.  
  
While the institutional investors are Lucas asset pricers, they have some dispersion of their fair price. This is set by the inst_val_std parameter. We want to see how that dispersion affect the prices and trading.  
  
## Setup  


## Static parameters

### SHARKFin
 --quarters 4
 --expectations InferentialExpectations
 --expectations ClientRPCMarket

(use the default CRRA, DiscFac, dividend growth rate and dividend std, p1, p2, AttentionSimulation, LUCAS0 population)


### AMMPS
 -days parameter to 240 to simulate a "sharkfin year".
 Grid - Variation:


Less MM Lucas factor, more II variation, more attention, higher zeta, higher dphm all contribute to the collapse of the financial system.
Other notes

We will continue to use the stripped down version of AMMPs with only MM, II, the Broker, and noise traders. This should have 30 minute runtime, but won't reproduce stylized facts of the stock market.

### AMMPS:  
As usual get the latest binaries from the master branch of ammps_sharkfin_container  
  
AMMPS comes with a number of new runtime options for outputs.  
  
The most important is the -c option, if it is set to true the logger write a compressed csv file. This saves a large amount of space and avoids us having to compress later. This comes with a small overhead on the cpu but if we are aiming for 1.1 core per simulation we should be covered.  
  
A second new option that might be useful is the -p option that adds a prefix to the logs written by ammps.  
  
For running a Lucas shark simulation with ammps only, we need to set an option to simulate a connection with SHARKfin this is done by setting the -s option to true. This option also comes with the -v and -d options that control the daily volume passed to the broker institutions simulating volume from the consumers in SHARKFin. In this case we just set both to 0.0. When using this option there should be no option to specify a RPC host etc.  
  
For running a Lucas Shark 0 simulation with ammps only we need to run:  
  
`dotnet [path_to]amm.engine.dll RunConfFromFile config_file output_directory -n unique_id_number [-c true] -s true -v 0.0 -d 0.0 [-p prefix]`  
  
I suggest using the compression option. Prefix option can be left out if you don't think it is usefull.  
  
### AMMPS Configuration Generator  
The ammps configuration generator generates a configuration file for the specific ammps simulation, this contains simulation specific parameters such as the period to be simulated, the random seed to be used, and the configurations for the agents in the simulations. It is now packed as a python package that can be installed in a python environment, or the scripts can be run using python.  
  
Scripts for generating the configs is found in the acg/simulations subdir.  
  
For the Lucas shark tests we need to run  
  
`make_lucas_shark_config.py [-h] [--out-dir CONF_DIR] --name RUN_NAME --seed SEED [--days NUMBER_OF_DAYS] --mm_lucas_factor MM_LUCAS_FACTOR [--mm_size MM_SIZE] [--inst_val_std DIVIDEND_VAL_STD]`  
  
The new options are --mm_lucas_factor MM_LUCAS_FACTOR [--mm_size MM_SIZE] [--inst_val_std DIVIDEND_VAL_STD]  
  
### Simulation Grid  
We want to test a simulation grid where we change mm_lucas_factor and inst_val_std, attention, dphm, zeta, pop_aNrmInitMean and seed.  
  

### AMMPS parameters:

  mm_lucas_factor =[0.0, 0.05, 0.15, 1.0]
  inst_val_std = [0.1,0.15, 0.2]

### SHARKFin parameters:

 attention = [0.005, 0.015, 0.05]
 dphm = [1000, 5000, 10000]
 zeta = [0, 0.3, .7, 1.0]
 pop_aNrmInitMean =  [5, 6, 7]




## Configure Parameters

In [ ]:
from simRunner import *
from simConfigs import *
#experiment parameters
experimentName='sparkshark'
tag ='rpc'
quarters=4
#Replicant Parameters:
seedcount=20
seedKey=6174
mmLucasFactors =[0.0, 0.05, 0.15, 1.0] 
instValStds = [0.1, 0.15, 0.2]
attention_values = [0.005, 0.015, 0.05]
dphms = [1000, 5000, 10000]
zetas = [0, 0.3, .7, 1.0]
pop_aNrmInitMeans =  [5, 6, 7]
rpc_host = '20.230.0.191'
quarters=4
cyclecloud='13.92.241.145'
finsim = '13.68.182.99'
user = 'ammpssharkfin'
simengine = 'simEngine.pem'

In [ ]:
print(experimentName)
#!az login
#!Connect-AzAccount 
#!az login --use-device-code`

## Build Simulation Objects

In [ ]:
#create entities in python list

simEntities = build_lucasShark_configs(experimentName,seedcount,seedKey,rpc_host,mmLucasFactors,instValStds,attention_values,dphms,zetas,quarters,tag)
simcount = len(simEntities)


## Load Simulation Objects in Azure Tables

In [ ]:
simcount

In [ ]:
#!az login
vaultName = 'sharkfinkv'
#names of needed secerts and keys - These can be located/updated here  ----> sharkfinkv.vault.azure.net
connectionName = 'simulationdataConnectionString'
#create table
newTable = create_table(experimentName,vaultName,connectionName)
#create a simulation grid clss object to store the simulation configurations
simTable = simulationGrid(experimentName, vaultName, connectionName)


In [ ]:
#create entities in Azure Table (csv list)
for sim in simEntities:
    create_table_entity(simTable,sim)

In [ ]:
sim1 = list(iter(simTable.simulations))

In [ ]:
getSim(experimentName,'1')

### View the simulations as a Datafram using the following cell.

In [ ]:

#retrives entities from Table
simgrid = simulationGrid(experimentName,vaultName,connectionName)
entities = simgrid.table_client.list_entities()
tableEntities = list(entities)
print(f"Located {len(tableEntities)} entities.")

edf = pd.DataFrame(tableEntities)
print(edf.info())
print(edf.head())

for key in tableEntities[1]:
    print(f"{key}={tableEntities[2][key]}")


In [ ]:
edf.to_csv('/Users/wjt5121/SharkSIM/lucasShark10parametergrid.csv')

In [ ]:
edf = pd.DataFrame(simEntities)
print(edf.info())
print(edf.head())
#display item details from Azure table
#edf.loc[2]['start_sharkfin_cmd']
#edf.loc[edf['simid'] == 2]


In [ ]:
experimentName='lucasShark10'
getSim(experimentName,'1')

In [ ]:
os.getcwd()

## Connect/Verify Slurm Cluster size/availabilty

* **CycleServer: https://13.92.241.145/home **
    * [Username/Passord: ammpssharkfin ammpsshrk1qaz@WSX
  PEM key is simEngine.pem  
Update the address and PEM key for your scheduler and in the next cell and test communication.


In [ ]:
cyclecloud='13.92.241.145'
finsim = '74.235.18.47'
user = 'ammpssharkfin'
simengine = 'simEngine.pem'
toolspath = 'copy_files_to_blob.py'

## Push node configuration and Requirements files to the Slurm scheduler
###Nodes are configured with a start-up script which is stored on the CucleCloud server in this location. -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/scripts) 

The requirments.txt file is located here -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/files/requirments.txt

In [ ]:

sbatchLocalPath = f'{experimentName}job.sh'
sbatchRemotePath = f'/shared/home/ammpssharkfin/{experimentName}job.sh'

runnerLocalPath = 'slurmRunnerLucasSharkStats.py'
runnerRemotePath = '/shared/home/ammpssharkfin/slurmRunnerstats.py'

clusterInitLocalPath = ''
clusterInitPath = '/opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/scripts/'
clusterInitLogPath = '/opt/cycle/jetpack/logs/cluster-init/slurm/default/scripts/'
requirementsLocalPath = 'requirements.txt'
requirmentsPath = '/shared/home/ammpssharkfin/'

configFile = '01-os-config-alma-linux.sh'
simengine = 'simEngine.pem'

toolspath = 'copy_files_to_blob.py'
toolsRemotePath = '/shared/home/ammpssharkfin/copy_files_to_blob.py'


In [ ]:
#Runs the getScheduleStatus function to validate connectivity to the the scheduler.
getSchedulerStatus(finsim, simengine, user)

### Create the Output directories

In [ ]:
#outdir = f'{experimentName}_output'
outdir = 'output'
cmd = f'mkdir {outdir};mkdir {outdir}/logs;mkdir {outdir}/logs/slurm; mkdir {outdir}/logs/sharkfin; mkdir {outdir}/logs/ammps_conf; mkdir {outdir}/logs/ammps'
#create initial directory structure on new scheduler
#cmd = 'mkdir output;mkdir output/logs;mkdir output/logs/slurm; mkdir output/logs/sharkfin; mkdir output/logs/ammps_conf; mkdir output/logs/ammps'
#cmd = 'ls -R output'
print(cmd)
run_cmd_remote(finsim, user, simengine, cmd)

### Push SlurmRunner script to scheduler

In [ ]:
print(runnerLocalPath)
#copy approriate slurmRunner.py for experiment. sharkFin/ammps vs ammps vs sharkFin
sendslurm = submit_file_to_remote(finsim, user, simengine, runnerLocalPath, runnerRemotePath, execute=False)

### Copy tools if required

In [ ]:
print(toolspath)
#copy approriate slurmRunner.py for experiment. sharkFin/ammps vs ammps vs sharkFin
sendslurm = submit_file_to_remote(finsim, user, simengine, toolspath, toolsRemotePath, execute=False)

### Push Requirements File to Scheduler

In [ ]:
#copy requirements.txt file to share 
sendslurm = submit_file_to_remote(lucashark, user, simengine, requirementsLocalPath, requirmentsPath, execute=False)

# Verify connection to the Scheduler and submit simulations jobs to Slurm Scheduler

### Generate the 'sbatch' script to submit the jobs to Slurm Scheduler

In [ ]:
#this function will create a sbatch script job,simcount is the number of simulation in the array you would like to run. 
job_cmd = generate_slurm_job(experimentName, simcount)
print(job_cmd)
print(sbatchLocalPath)

### Push 'sbatch' script to scheduler and execute.

In [ ]:
#create sbatch script used to execute simulation on slurm clustrer. if execute option is True jobs will be submited. 
out, err = submit_file_to_remote(finsim, user, simengine, sbatchLocalPath, sbatchRemotePath, execute=True)

In [ ]:
if out:print(out)
if err:print(err)

#todo Scheduler init needs to have creation of output and logs directories.

-Azure CycleCloud Server - Provides the Compute in the form of Slurm Clusters
-Slurm HPC Clusters
-Azure Storage Account - Blob and Files

-Customization script to config cluster nodes (located here -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/scripts)
    -Linux VM w/ Python3.9 + requirements.txt 
    -requirments.txt is located here -> /opt/cycle_server/work/staging/projects/slurm/2.6.2/default/cluster-init/files/requirments.txt
    -Cloning of the follow git repos
        https://github.com/mesalas/ammps_sharkfin_container.git /usr/simulation/ammps_bin    
        https://github.com/mesalas/ammps_config_generator /usr/simulation/ammps_config_generator
        https://github.com/sbenthall/SHARKFin /usr/simulation/SHARKFin/
        https://github.com/econ-ark/HARK.git /usr/simulation/harkrepo
        Note we must recopy the hark files - (investigate checking out the branch'sudo -H cp -Rf /usr/simulation/harkrepo/HARK/ /shared/home/ammpssharkfin/.local/lib/python3.9/site-packages/



### Use the generate_slurm_job function to configure the slurm job script used issue the simulation jobs to the scheduler.

## Monitor simulation progress

## Process simulation output

### Load SimStat and ClassStats as Dataframes

In [ ]:
# Example usage
simID = 400
experimentName = 'lucasShark10'
container = 'lucashark10'
simStatsSuffix =  '_sim_stats.txt'
tag = 'rpc'
simStatsFile = f'{experimentName}{simID}-{tag}{simStatsSuffix}'

#print(classStatsFile)
classdf = load_simStats_file_from_container(container, simStatsFile)
#classdf.head()

## Visualize Results from Experiment Grid

In [ ]:

#retrives entities from Table
simgrid = simulationGrid(experimentName,vaultName,connectionName)
entities = simgrid.table_client.list_entities()
tableEntities = list(entities)
print(f"Located {len(tableEntities)} entities. Loading dataframe")

edf = pd.DataFrame(tableEntities)
print(edf.info())
print(edf.head())


In [ ]:
simgrid.__dict__

### Display values for a simulation

In [ ]:
tableEntities